In [95]:
import re
import pandas as pd
import csv
import numpy as np
from collections import Counter
import nltk
import random
from nltk import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import IsolationForest


In [96]:
#read the csv file from the parser.py which uses regex to modify the data
#Clean it from the unwanted numbers such as timestamp, pid and len and time with the status codes
df=pd.read_csv('novaparse.csv')
#pd.set_option("display.max_rows", None, "display.max_columns", None)
df.head(5)

,Content
0,verbose level is INFO Request type is GET Stat...
1,verbose level is INFO Request type is GET Stat...
2,verbose level is INFO Request type is GET Stat...
3,verbose level is INFO Request type is GET Stat...
4,verbose level is INFO Request type is GET Stat...


In [97]:
df.shape

(515, 1)

In [98]:
#This is the second step of data cleaning where the special characters and extra white spaces are removed
df['Content']=df['Content'].str.replace(r'\W', " ", regex=True)
df['Content']=df['Content'].str.replace(r'  '," ", regex=True)
df = df[df['Content'].str.split().str.len().gt(5)]  
df.to_csv('test.csv')
#print(df)


In [99]:
df.shape

(398, 1)

In [100]:
df.head(5)

,Content
0,verbose level is INFO Request type is GET Stat...
1,verbose level is INFO Request type is GET Stat...
2,verbose level is INFO Request type is GET Stat...
3,verbose level is INFO Request type is GET Stat...
4,verbose level is INFO Request type is GET Stat...


In [101]:
type(df['Content'][0])

str

In [102]:
#Third step is to convert all the sentences into a lower case
#https://neptune.ai/blog/tokenization-in-nlp
df['Content']=df['Content'].str.lower()
#df['Content']=df['Content'].str.split()
df.to_csv('testtoke.csv')
df['Content']=df['Content'].apply(word_tokenize)
type(df['Content'][0])

list

In [103]:
df['Content'].head(5)

0    [verbose, level, is, info, request, type, is, ...
1    [verbose, level, is, info, request, type, is, ...
2    [verbose, level, is, info, request, type, is, ...
3    [verbose, level, is, info, request, type, is, ...
4    [verbose, level, is, info, request, type, is, ...
Name: Content, dtype: object

In [104]:
#tokens generated are saved in a file
#y.to_csv('tokens.csv')

In [105]:
#Remove the stopwords from the nltk stopword list
nltk.download('stopwords')
import nltk
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dramakrishna/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [106]:
#These are the stop words available, But lets create a custom stopwords for our convinience.
#As you can see that there are some single letters that will remove the letters
#from the words which will cause the word to lose its meaning
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [107]:
customlist = ['is','py','i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing',
              'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very'
              'can', 'will', 'just', "don't", 'should', "should've", 'now', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

In [108]:
#Removes the stop words
new=[word for word in df['Content'] if word not in customlist]
new[0]

['verbose',
 'level',
 'is',
 'info',
 'request',
 'type',
 'is',
 'get',
 'status',
 'code',
 'is',
 '200']

In [109]:
#For our reference lets make a count of each word
all_words=[]
for i in df['Content']:
    for j in i:
        all_words.append(j)

In [110]:
Counter(all_words)

Counter({'verbose': 275,
         'level': 275,
         'is': 826,
         'info': 275,
         'request': 276,
         'type': 274,
         'get': 274,
         'status': 274,
         'code': 274,
         '200': 274,
         'usr': 123,
         'lib': 123,
         'python': 123,
         'dist': 123,
         'packages': 123,
         'oslo_policy': 123,
         'policy': 248,
         'py': 123,
         'userwarning': 123,
         'os_compute_api': 244,
         'os': 238,
         'security': 28,
         'groups': 28,
         'rule': 242,
         'admin_or_owner': 56,
         'was': 123,
         'deprecated': 121,
         'in': 245,
         'favor': 121,
         'of': 121,
         'add': 5,
         'system_admin_or_owner': 32,
         'reason': 121,
         'services': 20,
         'admin_api': 65,
         'update': 7,
         'system_admin_api': 18,
         'delete': 27,
         'agents': 32,
         'list': 32,
         'system_reader_api': 47,
      

In [111]:
df.shape

(398, 1)

In [112]:
df['Content'][0]

['verbose',
 'level',
 'is',
 'info',
 'request',
 'type',
 'is',
 'get',
 'status',
 'code',
 'is',
 '200']

In [113]:
type(df['Content'][0])

list

In [114]:
#Pad the sentences with 0's depending upon the max length to keep the sentences consistent in length.
#Choosing a padding length of 50
header = ['Content']

with open('padded.csv', 'w', encoding='UTF8') as f:
  dw = csv.DictWriter(f, fieldnames=header)
  dw.writeheader()

  empty=[]
  j=[]
  for i in new:
      i_len = len(i)
      j=np.append(i, np.zeros(50-i_len))
      empty.append(j)
      print(empty, file=f)
f.close()

In [115]:
empty[238]

array(['usr', 'lib', 'python', 'dist', 'packages', 'oslo_policy',
       'policy', 'py', 'userwarning', 'policy', 'os_compute_api', 'os',
       'hypervisors', 'rule', 'admin_api', 'was', 'deprecated', 'in',
       'in', 'favor', 'of', 'os_compute_api', 'os', 'hypervisors',
       'uptime', 'rule', 'system_reader_api', 'reason', '0.0', '0.0',
       '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0',
       '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0',
       '0.0', '0.0'], dtype='<U32')

In [116]:
print(len(empty[238]))
empty[0][0]

50


'verbose'

In [117]:
onehot_enc = OneHotEncoder(sparse=False)
X = onehot_enc.fit(empty)
X = onehot_enc.transform(empty)

In [118]:
X[0]

array([0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0.,
       1., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
       1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 1., 1., 1.])

In [119]:
X

array([[0., 1., 1., ..., 1., 1., 1.],
       [0., 1., 1., ..., 1., 1., 1.],
       [0., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 0., 0., ..., 1., 1., 1.],
       [1., 0., 0., ..., 1., 1., 1.],
       [1., 0., 0., ..., 1., 1., 1.]])

In [120]:
#onehot_enc.inverse_transform(X)

In [121]:
#total features which includes 0;s
print(len(X[0]))

218


In [122]:
#Shuffle the data to randomly pick the rows for training and testing
random.shuffle(X)

In [146]:
#Splitting the data to train and test

split_len = 0.4 * len(X)
split_len
t1 = X[:int(split_len)]
X_train2 = X[int(split_len):]

In [147]:
t1[0]

array([0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0.,
       1., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
       1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 1., 1., 1.])

In [152]:
split_len2 = 0.8 * len(X_train2)
t2 = X_train2[:int(split_len)]
test = X_train2[int(split_len):]

In [153]:
t1.shape

(159, 218)

In [154]:
t2.shape

(159, 218)

In [155]:
test.shape

(80, 218)

In [156]:
X_test = pd.DataFrame(t1)
X_test.to_csv('X_train2_oh.csv')

In [157]:
X_train = pd.DataFrame(t2)
X_train.to_csv('X_train1_oh.csv')

In [158]:
X_test = pd.DataFrame(test)
X_test.to_csv('X_test_oh.csv')